In [23]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

In [24]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter working directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

# Specifying where to save plots
savepath = '/Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Animations_Background_Images/presentation_plots/'

Enter working directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m10q_res250/


In [25]:
# Get all rendered indices

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')
rendered_indices = rendered_df['snap'].to_list()
redshifts = rendered_df['redshift'].to_list()
num_snaps = len(rendered_indices)

# Decide desired metal

m = 'fe'
spath_m = sdir + m + '/'

In [26]:
times = np.array(rendered_df['time'])
delta_times = np.max(times) - np.min(times)
times_range = np.arange(np.min(times), np.max(times), 0.01)

fit_params_df = pd.read_csv(spath_m + 'data/fit/fit_{}_params.csv'.format(m))

spreads = np.array(fit_params_df['std'])
delta_spreads = np.max(spreads) - np.min(spreads)
spreads_range = np.arange(np.min(spreads), np.max(spreads), 0.01)

medians = np.array(fit_params_df['median_mass'])
delta_medians = np.max(medians) - np.min(medians)
medians_range = np.arange(np.min(medians), np.max(medians), 0.01)

mach_numbers = np.array(rendered_df['mach_number_mass'])

In [27]:
def exp_fit(X, A, alpha):
    Y = A*np.exp(alpha*X)
    return Y

In [28]:
eps = 0.005

In [22]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize = (25, 30), sharex = True)

# Plot median

ax = axs[0]

B = np.max(medians) + eps

shift_times = times - np.min(times)
shift_medians = medians - B
shift_medians_ratio = shift_medians[len(shift_medians)-1]/shift_medians[0]

A = shift_medians[0]
alpha = np.log(shift_medians_ratio)/delta_times

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_times, shift_medians, p0 = guess_params)
print(fit_params)

shift_times_range = np.arange(np.min(shift_times), np.max(shift_times), 0.01)
shift_fit_medians = exp_fit(shift_times_range, *fit_params)
fit_medians = shift_fit_medians + B

ax.scatter(times, medians, s = 100, label = 'Simulation', color = 'green', alpha = 0.5)
ax.plot(times_range, fit_medians, color = 'green', lw = 8, label = 'Analytic Fit')

ax.set_ylabel('Median Abundance', labelpad = 10, fontsize = 74)
ax.set_title('[{}/H]'.format(m.title()), fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

ax.legend(prop={'size': 74})

# Plot spread

ax = axs[1]

B = np.min(spreads) - eps

shift_times = times - np.min(times)
shift_spreads = spreads - B
shift_spreads_ratio = shift_spreads[len(shift_spreads)-1]/shift_spreads[0]

A = shift_spreads[0]
alpha = np.log(shift_spreads_ratio)/delta_times

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_times, shift_spreads, p0 = guess_params)
print(fit_params)

shift_times_range = np.arange(np.min(shift_times), np.max(shift_times), 0.01)
shift_fit_spreads = exp_fit(shift_times_range, *fit_params)
fit_spreads = shift_fit_spreads + B

ax.scatter(times, spreads, s = 100, color = 'green', alpha = 0.5)
ax.plot(times_range, fit_spreads, color = 'green', lw = 8)

ax.set_xlabel('Time (Gyr)', labelpad = 10, fontsize = 74)
ax.set_ylabel('Abundance Spread', labelpad = 72, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

# Save file

plt.savefig(savepath + 'abundance_time.png', bbox_inches='tight')
plt.close()

[-1.84921622 -0.43793038]
[ 0.1776432  -0.00232782]


In [29]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize = (25, 30), sharex = True)

# Turbulence and median abundance
ax = axs[0]

B = np.min(mach_numbers) - eps

shift_medians = medians - np.min(medians)
shift_mach_numbers = mach_numbers - B
shift_mach_numbers_ratio = shift_mach_numbers[len(shift_mach_numbers)-1]/shift_mach_numbers[0]

A = shift_mach_numbers[0]
alpha = np.log(shift_mach_numbers_ratio)/delta_medians

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_medians, shift_mach_numbers, p0 = guess_params)
print(fit_params)

shift_medians_range = np.arange(np.min(shift_medians), np.max(shift_medians), 0.01)
shift_fit_mach_numbers = exp_fit(shift_medians_range, *fit_params)
fit_mach_numbers = shift_fit_mach_numbers + B

ax.scatter(mach_numbers, medians, s = 100, label = 'Simulation', color = 'red', alpha = 0.5)
ax.plot(fit_mach_numbers, medians_range, color = 'red', lw = 8, label = 'Analytic Fit')

ax.set_ylabel('Median Abundance', labelpad = 10, fontsize = 74)
ax.set_title('[{}/H]'.format(m.title()), fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

ax.legend(prop={'size': 74})

# Turbulence and abundance spread

ax = axs[1]

B = np.min(mach_numbers) - eps

shift_spreads = spreads - np.min(spreads)
shift_mach_numbers = mach_numbers - B
shift_mach_numbers_ratio = shift_mach_numbers[len(shift_mach_numbers)-1]/shift_mach_numbers[0]


A = shift_mach_numbers[0]
alpha = np.log(shift_mach_numbers_ratio)/delta_spreads

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_spreads, shift_mach_numbers, p0 = guess_params)
print(fit_params)

shift_spreads_range = np.arange(np.min(shift_spreads), np.max(shift_spreads), 0.01)
shift_fit_mach_numbers = exp_fit(shift_spreads_range, *fit_params)
fit_mach_numbers = shift_fit_mach_numbers + B

ax.scatter(mach_numbers, spreads, s = 100, color = 'red', alpha = 0.5)
ax.plot(fit_mach_numbers, spreads_range, color = 'red', lw = 8)

ax.set_xlabel('Mean Mach-Number', labelpad = 10, fontsize = 74)
ax.set_ylabel('Abundance Spread', labelpad = 72, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

plt.savefig(savepath + 'turbulence-abundance.png', bbox_inches='tight')
plt.close()

[0.04190219 3.04975294]
[  6.41687156 -11.50974455]
